## Recursive training of parameters
Let us consider a certain distribution for which we would like to train the parameters.
Let $\mathbf{Y}$ be the $q$ dimensional response variable, and let $\mathbf{X}$ be the vector of $k$ explanatory variables. We have
$$
\mathbf{Y} | \mathbf{X} = \mathbf{x} \sim  \mathcal{N}_q(\Theta^t \cdot \mathbf{x}, \Gamma)
$$
with $\Theta$ a $k\times q$ matrix and $\Gamma$ a $q\times q$ covariance matrix.

Now, the objective is to update parameters $\Theta$ and $\Gamma$ recursively. Let us suppose that we already have estimates for these parameters, and that new observations $\mathbf{y}$ for $\mathbf{Y}$ and $\mathbf{x}$ for $\mathbf{X}$ become available.

First we update the so-called "support structures". For the $n$-th step the formulas are the following:
- $H_n = \mathbf{x}_n \mathbf{y}_n^t + \lambda H_{n-1}$
- $J_n = \mathbf{x}_n \mathbf{x}_n^t + \lambda J_{n-1}$
- $K_n = \mathbf{y}_n \mathbf{y}_n^t + \lambda K_{n-1}$
- $\gamma_n = 1 + \lambda \gamma_{n-1}$

and at the start for $n=0$, all four support structures are initialized as zero-matrices.

Then we update the values for the required parameters using these formulas:
- $\hat{\Theta}_n = (J_n)^{-1} H_n$
- $\hat{\Gamma}_n = \frac{1}{\gamma_n} \left( K_n - H_n^t \hat{\Theta}_n \right)$.

Notice that during training, if we don't need to to the prediction we do not need to calculate $\hat{\Theta}_n$ and $\hat{\Gamma}_n$.

---

## Recursive predictions

Let us define:
- The **transition distribution** is: <br/> $\mathbf{X}_k | \mathbf{X}_{k-1}=\mathbf{x}_{k-1} \sim \mathcal{N}_d ( A_{c(k)} \mathbf{x}_{k-1} + \boldsymbol{\alpha}_{c(k)} , Q_{c(k)})$
- The **emission distribution** is: <br/> $\mathbf{Y}_k | \mathbf{X}_k=\mathbf{x}_k \sim \mathcal{N}_p ( B_{c(k)} \mathbf{x}_k + \boldsymbol{\beta}_{c(k)} , R_{c(k)})$
- The **emission distribution for MAPLF** is: <br/> $\mathbf{X}_k | \mathbf{Y}_k=\mathbf{y}_k \sim \mathcal{N}_d ( D_{c(k)} \mathbf{y}_k + \boldsymbol{\delta}_{c(k)} , P_{c(k)})$
- The **distribution for Inverted State-Space Model** is: <br/> $\begin{bmatrix}\mathbf{X}_k\\ \mathbf{Y}_k\end{bmatrix}| \mathbf{X}_{k-1}=\mathbf{x}_{k-1} \sim \mathcal{N}_{d+p} \left( \begin{bmatrix}A_{c(k)}\\ E_{c(k)}\end{bmatrix}\mathbf{x}_{k-1} + \begin{bmatrix}\boldsymbol{\alpha}_{c(k)}\\ \boldsymbol{\epsilon}_{c(k)}\end{bmatrix}, \begin{bmatrix}Q_{c(k)} & U_{c(k)}\\ U_{c(k)}^t & S_{c(k)}\end{bmatrix}\right)$

where $c$ is a function that maps each time step to the corresponding calendar type (e.g. 5am of a weekend)

The predictions are done recursively. For $k=1:horizon,$ the following formulas are used for the different forecasting algorithms.

When $k=0$, $\hat{\Sigma}_{k}$ is a zero-matrix and $\hat{\boldsymbol{\mu}}_{k}$ is equal to the real observation during that time.

### MAPLF:

The covariance is
$$
\hat{\Sigma}_{k} = \Big(\big(Q_{c(k)} + A_{c(k)}\hat{\Sigma}_{k-1} A_{c(k)}^t\big)^{-1}+P_{c(k)}^{-1}\Big)^{-1}
$$

and the mean is
$$
\hat{\boldsymbol{\mu}}_{k} = \hat{\Sigma}_{k}\Big[\big(Q_{c(k)} + A_{c(k)}\hat{\Sigma}_{k-1} A_{c(k)}^t\big)^{-1}(A_{c(k)}\hat{\boldsymbol{\mu}}_{k-1} + \boldsymbol{\alpha}_{c(k)}) + P_{c(k)}^{-1}(D_{c(k)}\mathbf{y}_{k} + \boldsymbol{\delta}_{c(k)}) \Big]
$$

### Kalman Filter:

The covariance is
$$
\hat{\Sigma}_{k} = \Big(\big(Q_{c(k)} + A_{c(k)}\hat{\Sigma}_{k-1}A_{c(k)}^t\big)^{-1} + B_{c(k)}^tR_{c(k)}^{-1}B_{c(k)}\Big)^{-1}
$$

and the mean is
$$
\hat{\boldsymbol{\mu}}_{k} = \hat{\Sigma}_{k}\Big[ \big(Q_{c(k)} + A_{c(k)}\hat{\Sigma}_{k-1} A_{c(k)}^t\big)^{-1}(A_{c(k)}\hat{\boldsymbol{\mu}}_{k-1} + \boldsymbol{\alpha}_{c(k)}) + B_{c(k)}^t R_{c(k)}^{-1}\big(\mathbf{y}_{k} - \boldsymbol{\beta}_{c(k)}\big)\Big]
$$

### Inverted State-Space Model

The covariance is
$$
\hat{\Sigma}_{k} = (Q_{c(k)} + A_{c(k)}\hat{\Sigma}_{k-1}A_{c(k)}^t) - (U_{c(k)}+A_{c(k)}\hat{\Sigma}_{k-1}E_{c(k)}^t)(S_{c(k)}+E_{c(k)}\hat{\Sigma}_{k-1}E_{c(k)}^t)^{-1} (U_{c(k)}^t +E_{c(k)}\hat{\Sigma}_{k-1}A_{c(k)}^t)
$$

and the mean is
$$
\hat{\boldsymbol{\mu}}_{k} = A_{c(k)}\hat{\boldsymbol{\mu}}_{k-1} + \boldsymbol{\alpha}_{c(k)} + (U_{c(k)}+A_{c(k)}\hat{\Sigma}_{k-1}E_{c(k)}^t)(S_{c(k)}+E_{c(k)}\hat{\Sigma}_{k-1}E_{c(k)}^t)^{-1}(\mathbf{y}_{k} - E_{c(k)}\hat{\boldsymbol{\mu}}_{k-1}- \boldsymbol{\epsilon}_{c(k)})
$$

### VAR Model

The covariance is
$$
\hat{\Sigma}_{k} = Q_{c(k)} + A_{c(k)}\hat{\Sigma}_{k-1}A_{c(k)}^t
$$

and the mean is
$$
\hat{\boldsymbol{\mu}}_{k} = A_{c(k)}\hat{\boldsymbol{\mu}}_{k-1} + \boldsymbol{\alpha}_{c(k)}
$$